In [56]:
# Notices that MP2 works with fermionic 1e and 2e tensors in Physicist notation, in spin orbital basis
import sys
sys.path.append("../")
import pickle
import io
import saveload_utils as sl
import ferm_utils as feru
import csa_utils as csau
import var_utils as varu
import openfermion as of
import numpy as np
import scipy
from openfermion import FermionOperator
import copy
from itertools import product
from fstate import *
from openfermion.chem import MolecularData

mol = 'h2'
tol = 1e-5
steps = 5

def MP2_correction(atbt, obt_eigs, state):
    n = len(state)
    occupied = [i for i in range(n) if state[i] == "1"]
    virtual = [i for i in range(n) if state[i] == "0"]
    tmp = 0
    for i,j in product(occupied, repeat = 2):
        for a, b in product(virtual, repeat = 2):
            k = obt_eigs[i] + obt_eigs[j] - obt_eigs[a] - obt_eigs[b]
            if abs(k) > 1e-14:
                tmp += atbt[i,j,a,b] ** 2 / k
    return tmp/4

result = {}
# Get two-body tensor
for mol in ["h2", "h4", "lih"]:
    Hf = sl.load_fermionic_hamiltonian(mol)
    n =  of.count_qubits(Hf)  
    obt = feru.get_obt(Hf, n = n, spin_orb = True )
    tbt = feru.get_two_body_tensor(Hf, n = n)
    obt_eigs, _ = np.linalg.eig(obt)
    # Construct antisymmetric 
    atbt = np.zeros(tbt.shape)
    for i,j,k,l in product(range(n), repeat = 4):
        atbt[i,j,k,l] = tbt[i,j,k,l] - tbt[i,j,l,k]
    tmp = {}

    # Construct Chemist notation Htbt
    Htbt = feru.get_chemist_tbt(Hf, n, spin_orb = True)
    one_body = varu.get_one_body_correction_from_tbt(Hf, feru.get_chemist_tbt(Hf))
    onebody_matrix = feru.get_obt(one_body, n = n, spin_orb = True)
    Htbt += feru.onebody_to_twobody(onebody_matrix)
    # FCI computation, takes exponential time and space !!!
    const = Hf.terms[()]
    eigenvalues = of.linalg.eigenspectrum(Hf)
    E_max = max(eigenvalues) - const
    E_min = min(eigenvalues) - const
    print("E_max: {}".format(E_max))
    print("E_min: {}".format(E_min))
    dE = E_max - E_min
    print("dE: {}".format(dE))
    tmp["E_max"] = E_max
    tmp["E_min"] = E_min
    tmp["dE"] = dE

    # Run HF and MP2, using results from HF
    max_state, min_state, HF_max, HF_min = HF_spectrum_range(Htbt, multiprocessing = True)
    tmp["HF_max"] = HF_max
    tmp["HF_min"] = HF_min
    print(max_state, min_state)
    MP2_min = HF_min + MP2_correction(atbt, obt_eigs, min_state)
    MP2_max = HF_max + MP2_correction(atbt, obt_eigs, max_state)
    print("MP2 E_max: {}".format(MP2_max))
    print("MP2 E_min: {}".format(MP2_min))
    tmp["MP2_max"] = MP2_max
    tmp["MP2_min"] = MP2_min

    dE_HF = HF_max - HF_min
    dE_MP2 = MP2_max - MP2_min
    print("HF dE: {}".format(dE_HF))
    print("MP2 dE: {}".format(dE_MP2))
    tmp["dE_HF"] = dE_HF
    tmp["dE_MP2"] = dE_MP2

    # Run Lanczos, for interation number = steps
    steps = 3
    lanczos_max, lanczos_min = lanczos_total_range(Htbt, steps = steps, 
                                                   states = [max_state, min_state])
    tmp["lanczos_max"] = lanczos_max
    tmp["lanczos_min"] = lanczos_min
    dE_lanczos = lanczos_max - lanczos_min
    tmp["dE_lanczos"] = dE_lanczos
    print(tmp)
    result[mol] = tmp


E_max: 0.0
E_min: -1.6303275411526186
dE: 1.6303275411526186
HF E_max: 0
HF E_min: -1.595285860237936
0000 1100
MP2 E_max: 0.0
MP2 E_min: -1.6045643760095274
HF dE: 1.595285860237936
MP2 dE: 1.6045643760095274
{'E_max': 0.0, 'E_min': -1.6303275411526186, 'dE': 1.6303275411526186, 'HF_max': 0, 'HF_min': -1.595285860237936, 'MP2_max': 0.0, 'MP2_min': -1.6045643760095274, 'dE_HF': 1.595285860237936, 'dE_MP2': 1.6045643760095274, 'lanczos_max': 0.0, 'lanczos_min': -1.6303275411526181, 'dE_lanczos': 1.6303275411526181}
E_max: 0.5774596978922335
E_min: -4.459488695954763
dE: 5.0369483938469966
HF E_max: 0
HF E_min: -4.391647184317761
00000000 11110000
MP2 E_max: 0.0
MP2 E_min: -4.387871114492357
HF dE: 4.391647184317761
MP2 dE: 4.387871114492357
{'E_max': 0.5774596978922335, 'E_min': -4.459488695954763, 'dE': 5.0369483938469966, 'HF_max': 0, 'HF_min': -4.391647184317761, 'MP2_max': 0.0, 'MP2_min': -4.387871114492357, 'dE_HF': 4.391647184317761, 'dE_MP2': 4.387871114492357, 'lanczos_max': 0.0

KeyboardInterrupt: 

In [55]:
for mol in result:
    tmp = result[mol]
    print(mol)
    print("dE: {}".format(tmp["dE"]))
    print("HF dE: {}".format(tmp["dE_HF"]))
    print("MP2 dE: {}".format(tmp["dE_MP2"]))
    print("Lancoz-3 dE: {}".format(tmp["dE_lanczos"]))

h2
dE: 1.6303275411526186
HF dE: 1.595285860237936
MP2 dE: 1.6045643760095274
Lancoz-3 dE: 1.6303275411526181
h4
dE: 5.0369483938469966
HF dE: 4.391647184317761
MP2 dE: 4.387871114492357
Lancoz-3 dE: 4.457453599273439
lih
dE: 9.865763593799164
HF dE: 9.833079281109747
MP2 dE: 9.833331702422104
Lancoz-3 dE: 9.955068181293232
